LSTM

Import packages.

In [2]:
import torch
import os
import csv
import random
import numpy as np
import pandas as pd
import torch.nn.functional as F
import keras
import tensorflow as tf

from torch.nn.utils.rnn import pad_sequence
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

Download the dataset and unzip it.

In [3]:
!gdown 1wdlmKyBxd3_UdwOaNgfwx6mt6DMas6uS
!gdown 1E22shQHUmSvqkLPe6qGji4TIYArtOAKf
!gdown 141AattP4xJ3fm2n9ZWFh1wvSfwVOl7cT
!gdown 1XUa6TMZ_ytk3I_jQRJ0iC7tHQWUAZghw
!gdown 1FIMWCzeRN9C6NurXyq4NnnUc5W1DeMDk
!gdown 1n4haJzeyPJxPm2ukVBg1sM84CSmJimV8


!gdown 1RKM_eyCjuUPG50Kl6azWnkOaPkHObBld
!gdown 1IncHTNbn7GJ69tCHzKDRpNPTYKzX7Ias
!gdown 1dq2zpR9N0OXtqR-qhQE0ny5H1RT4oOG7
!gdown 16Ll_Q041wFe1PSGULeLph3oQSKH6_yNg
!gdown 1qhkb3-__1Ubl47Sicsfniy1JtDUXE0Zf
!gdown 1P6LT9FnLx3vuXcD7402dBhQIJQw5EwBo

Downloading...
From: https://drive.google.com/uc?id=1wdlmKyBxd3_UdwOaNgfwx6mt6DMas6uS
To: /content/train_y.csv
100% 180k/180k [00:00<00:00, 82.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1E22shQHUmSvqkLPe6qGji4TIYArtOAKf
To: /content/V_trade.csv
100% 7.28M/7.28M [00:00<00:00, 135MB/s]
Downloading...
From: https://drive.google.com/uc?id=141AattP4xJ3fm2n9ZWFh1wvSfwVOl7cT
To: /content/V_remit.csv
100% 2.80M/2.80M [00:00<00:00, 154MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XUa6TMZ_ytk3I_jQRJ0iC7tHQWUAZghw
To: /content/V_info.csv
100% 533k/533k [00:00<00:00, 118MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FIMWCzeRN9C6NurXyq4NnnUc5W1DeMDk
To: /content/V_cons.csv
100% 4.96M/4.96M [00:00<00:00, 25.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1n4haJzeyPJxPm2ukVBg1sM84CSmJimV8
To: /content/V_cred.csv
100% 13.5M/13.5M [00:00<00:00, 198MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RKM_eyCjuUPG50Kl6azWnkOaPkHObBld
To: /content/

Basic setup of hyperparameters

In [142]:
'''
BATCH_SIZE = 256
EPOCH_NUM = 50
MAX_POSITIONS_LEN = 100
SEED = 97562246875 % (2**32-1) # Set your lucky number as the random seed
MODEL_DIR = 'model.pth'
lr = 5 * 1e-4

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
random.seed(SEED)
np.random.seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

w2v_config = {'path': 'w2v.model', 'dim': 128}
net_config = {'hidden_dim': 64, 'num_layers': 3, 'bidirectional': False, 'fix_embedding': True}
header_config = {'dropout': 0.5, 'hidden_dim': 64}
assert header_config['hidden_dim'] == net_config['hidden_dim'] or header_config['hidden_dim'] == net_config['hidden_dim'] * 2
'''

Combine all V_ together

In [22]:
V_cred = pd.read_csv("V_cred.csv", index_col=0) 
V_info = pd.read_csv("V_info.csv", index_col=0)             
V_remit = pd.read_csv("V_remit.csv", index_col=0) 
V_cons = pd.read_csv("V_cons.csv", index_col=0)             
V_trade = pd.read_csv("V_trade.csv", index_col=0) 
train_y = pd.read_csv("train_y.csv", index_col=0)

V_cred_public = pd.read_csv("V_cred_public.csv", index_col=0) 
V_info_public = pd.read_csv("V_info_public.csv", index_col=0)             
V_remit_public = pd.read_csv("V_remit_public.csv", index_col=0) 
V_cons_public = pd.read_csv("V_cons_public.csv", index_col=0)             
V_trade_public = pd.read_csv("V_trade_public.csv", index_col=0) 
public_y = pd.read_csv("public_y.csv", index_col=0)

print(V_cred.shape)
print(V_info.shape)
print(V_remit.shape)
print(V_cons.shape)
print(V_trade.shape)
print(train_y.shape)

V_part = [ pd.concat([V_info, 
                      V_cred.iloc[:,8*i:8*(i+1)],
                      V_remit.iloc[:,3*i:3*(i+1)],
                      V_cons.iloc[:,4*i:4*(i+1)],
                      V_trade.iloc[:,7*i:7*(i+1)]], axis=1) for i in range (13)]
V_all = pd.concat([V_part[i] for i in range (13)], axis = 1)
V_all = (V_all-V_all.mean())/V_all.std()
print(V_all.shape) # put all data together (23906 * 338)

V_part_public = [ pd.concat([V_info_public, 
                      V_cred_public.iloc[:,8*i:8*(i+1)],
                      V_remit_public.iloc[:,3*i:3*(i+1)],
                      V_cons_public.iloc[:,4*i:4*(i+1)],
                      V_trade_public.iloc[:,7*i:7*(i+1)]], axis=1) for i in range (13)]
V_all_public = pd.concat([V_part_public[i] for i in range (13)], axis = 1)
V_all_public = (V_all_public-V_all_public.mean())/V_all_public.std()
print(V_all_public.shape) # put all data together (1845 * 338)


(23906, 104)
(23906, 4)
(23906, 39)
(23906, 52)
(23906, 91)
(23906, 1)
(23906, 338)
(1845, 338)


oversampling

In [ ]:
'''
# oversampling

SAR_idx = []
for i in range (len(train_Y)):
    if (train_Y.iloc[i,0] == 1):
        SAR_idx.append(i)
print(len(SAR_idx), SAR_idx)

for i in range (len(SAR_idx)):
    if (i % 50 == 0):
        print(i, "/", len(SAR_idx))
    for j in range (30):
        V_cred.loc[len(V_cred)] = V_remit.loc[SAR_idx[i]]
print(V_cred)
'''

In [23]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(units = 50, return_sequences = True, input_shape = (13,26)))
model.add(keras.layers.Dropout(0.2))
#model.add(keras.layers.LSTM(units = 50, return_sequences = True))
#model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(units = 50))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='nadam')

#model = keras.models.load_model('path/to/location')


tempx = np.array(V_all)
tempy = np.array(train_y)
tempx = np.reshape(tempx, (tempx.shape[0], 13, -1))
tempx[np.isnan(tempx)] = 0
print(tempx.shape)
model.fit(tempx, tempy, epochs = 50, batch_size = 32, class_weight = {0:1, 1:100})

model.save('model.pth')


(23906, 13, 26)
Epoch 1/50


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


748/748 [==============================] - 18s 18ms/step - loss: 1.2768
Epoch 2/50
748/748 [==============================] - 13s 18ms/step - loss: 1.1778
Epoch 3/50
748/748 [==============================] - 13s 18ms/step - loss: 1.1421
Epoch 4/50
748/748 [==============================] - 13s 17ms/step - loss: 1.1077
Epoch 5/50
748/748 [==============================] - 13s 17ms/step - loss: 1.0872
Epoch 6/50
748/748 [==============================] - 13s 18ms/step - loss: 1.0296
Epoch 7/50
748/748 [==============================] - 14s 18ms/step - loss: 1.0292
Epoch 8/50
748/748 [==============================] - 14s 19ms/step - loss: 0.9537
Epoch 9/50
748/748 [==============================] - 13s 17ms/step - loss: 0.9604
Epoch 10/50
748/748 [==============================] - 13s 17ms/step - loss: 0.9141
Epoch 11/50
748/748 [==============================] - 13s 18ms/step - loss: 0.8699
Epoch 12/50
748/748 [==============================] - 13s 17ms/step - loss: 0.8444
Epoch 13/50


In [24]:
def check_score(model, x, y):
    prediction = model.predict(x)
    print(prediction, prediction.shape)
    result = pd.concat([pd.DataFrame(prediction, columns=["prob"]), pd.DataFrame(y)], axis=1) 
    result = result.sort_values(by = 'prob')
    print(result)
    for i in range (len(result)):
        if (result.iloc[i,1] == 1):
            print("The", i, "th lowest prediction has SAR with prob", result.iloc[i,0])

!zip -r model.zip model.pth
from google.colab import files
files.download("model.zip")

check_score(model, tempx, tempy)
testx = np.array(V_all_public)
testy = np.array(public_y)
testx = np.reshape(testx, (testx.shape[0], 13, -1))
testx[np.isnan(testx)] = 0

SAR_idx = []
for i in range (len(public_y)):
    if (public_y.iloc[i,0] == 1):
        SAR_idx.append(i)
print(len(SAR_idx), SAR_idx)
check_score(model, testx, testy)

  adding: model.pth/ (stored 0%)
  adding: model.pth/variables/ (stored 0%)
  adding: model.pth/variables/variables.index (deflated 63%)
  adding: model.pth/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: model.pth/saved_model.pb (deflated 90%)
  adding: model.pth/assets/ (stored 0%)
  adding: model.pth/keras_metadata.pb (deflated 90%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

748/748 [==============================] - 5s 6ms/step
[[2.4444619e-04]
 [6.1140767e-05]
 [8.6536998e-01]
 ...
 [5.4440283e-05]
 [2.6745145e-04]
 [9.3435854e-05]] (23906, 1)
           prob  0
1913   0.000020  0
2092   0.000020  0
5731   0.000021  0
2305   0.000021  0
23789  0.000022  0
...         ... ..
8357   0.993268  1
6686   0.994810  1
7995   0.994894  1
14879  0.995880  1
17369  0.998028  1

[23906 rows x 2 columns]
The 19191 th lowest prediction has SAR with prob 0.25234562
The 19221 th lowest prediction has SAR with prob 0.25619867
The 19312 th lowest prediction has SAR with prob 0.26723936
The 19770 th lowest prediction has SAR with prob 0.337388
The 19940 th lowest prediction has SAR with prob 0.35705876
The 20187 th lowest prediction has SAR with prob 0.40423423
The 20189 th lowest prediction has SAR with prob 0.40484288
The 20561 th lowest prediction has SAR with prob 0.49177474
The 20785 th lowest prediction has SAR with prob 0.5343445
The 20845 th lowest prediction has 